In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product,chain, permutations
import gurobipy as gb
from gurobipy import GRB
from tqdm import tqdm

repeats = json.load(open('../cleanerdata/repeats.json'))
repeats = {int(k):v for k,v in repeats.items()}

locdem = pd.read_excel('../cleanerdata/locdem.xlsx')

q = locdem['Number of pallets'].to_numpy().astype(float).tolist()
q = [0] + q
q.extend([v['dem'] for v in repeats.values()])

locs = pd.read_excel('../cleanerdata/locations.xlsx')
longs, lats = locs['long'].to_numpy().tolist(), locs['lat'].to_numpy().tolist()
longs.extend([longs[v['map']] for v in repeats.values()])
lats.extend([lats[v['map']] for v in repeats.values()])

distmat = pd.read_json('../cleanerdata/distmat.json').to_numpy()
timemat = pd.read_json('../cleanerdata/timemat.json').to_numpy()
def get(i):
    try:
        i = repeats[i]['map']
    except KeyError:
        pass
    return i

reverse_mapping = defaultdict(list)
for k,v in repeats.items():
    reverse_mapping[v['map']].append(k)
for k in reverse_mapping:
    reverse_mapping[k].append(k)

mappings = list(chain(*[permutations(ls,2) for ls in reverse_mapping.values()]))

def cost(i,j):
    if (i,j) in mappings:
        return 1e+8
    i,j = get(i), get(j)
    return distmat[i,j]


def time(i,j):
    i,j = get(i), get(j)
    return timemat[i,j]

def location(i):
    return longs[i], lats[i]

In [2]:
clusters = json.load(open('./clusters.json'))
list(map(lambda x: len(x), clusters))

[5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5]

In [3]:
Q = 9
iter_count = 0
primary_models = []
Ls = []

for cluster in clusters:
    Neighbours = list(sorted(cluster))
    all_nodes = [0] + Neighbours.copy()
    v = len(all_nodes)
    V = list(range(v))
    N = V.copy()
    N.remove(0)
    
    def Del(i):
        F = V.copy()
        F.remove(i)
        return F
    
    m = gb.Model()
    x = m.addVars(v,v, vtype=GRB.BINARY, name='x')
    m.setObjective(sum([cost(all_nodes[i], all_nodes[j]) * x[i,j] for i,j in product(V,V)]))
    for i in N:
        m.addConstr(sum([x[i,j] for j in Del(i)]) == 1)
    for j in N:
        m.addConstr(sum([x[i,j] for i in Del(j)]) == 1)
        
    l = m.addVar(lb=0.0, vtype=GRB.INTEGER, name='L')
    Ls.append(l)
    m.addConstr(sum([x[0,j] for j in Del(0)]) == l)
    
    u = m.addVars(V, vtype=GRB.CONTINUOUS, name='u')
    for i,j in filter(lambda tup: tup[0] != tup[1], product(N,N)):
        m.addConstr(u[i] - u[j] + Q*x[i,j] <= Q - q[j])
    for i in V:
        m.addConstr(u[i] >= q[i])
        
    for pair in mappings:
        if pair[0] in cluster and pair[1] in cluster:
            pair = Neighbours.index(pair[0])+1, Neighbours.index(pair[1])+1
            print(Neighbours, pair)
            for length in range(1,6):
                all_groups = permutations(N,length)
                for group in all_groups:
                    if pair[0] in group or pair[1] in group:
                        continue
                    group = [pair[0]] + list(group) + [pair[1]]
                    edges = [(group[i], group[i+1]) for i in range(len(group) - 1)]
                    expr = 0
                    for edge in edges:
                        expr += x[edge]
                    m.addConstr(expr <= length)
    m.update()
    #print(m.display())
    primary_models.append(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12
[1, 45, 67, 105, 166] (5, 4)
[1, 45, 67, 105, 166] (4, 5)
[4, 40, 151, 152, 153] (3, 4)
[4, 40, 151, 152, 153] (3, 5)
[4, 40, 151, 152, 153] (3, 2)
[4, 40, 151, 152, 153] (4, 3)
[4, 40, 151, 152, 153] (4, 5)
[4, 40, 151, 152, 153] (4, 2)
[4, 40, 151, 152, 153] (5, 3)
[4, 40, 151, 152, 153] (5, 4)
[4, 40, 151, 152, 153] (5, 2)
[4, 40, 151, 152, 153] (2, 3)
[4, 40, 151, 152, 153] (2, 4)
[4, 40, 151, 152, 153] (2, 5)
[5, 25, 125, 126, 127] (3, 4)
[5, 25, 125, 126, 127] (3, 5)
[5, 25, 125, 126, 127] (3, 1)
[5, 25, 125, 126, 127] (4, 3)
[5, 25, 125, 126, 127] (4, 5)
[5, 25, 125, 126, 127] (4, 1)
[5, 25, 125, 126, 127] (5, 3)
[5, 25, 125, 126, 127] (5, 4)
[5, 25, 125, 126, 127] (5, 1)
[5, 25, 125, 126, 127] (1, 3)
[5, 25, 125, 126, 127] (1, 4)
[5, 25, 125, 126, 127] (1, 5)
[6, 89, 103, 128, 130] (4, 5)
[6, 89, 103, 128, 130] (4, 1)
[6, 89, 103, 128, 130] (5, 4)
[6, 89, 103, 128, 130] (5, 1)
[6, 89, 103

In [4]:
for model in tqdm(primary_models):
    model.optimize();

  0%|                                                    | 0/36 [00:00<?, ?it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 67 rows, 43 columns and 218 nonzeros
Model fingerprint: 0x910afe71
Variable types: 6 continuous, 37 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+04, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 336799.00000
Presolve removed 7 rows and 20 columns
Presolve time: 0.01s
Presolved: 60 rows, 23 columns, 226 nonzeros
Variable types: 5 continuous, 18 integer (18 binary)
Found heuristic solution: objective 224511.00000

Root relaxation: objective 1.502397e+05, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 150239.667    0    7 224511.000 150239.667  33.1

  3%|█▏                                          | 1/36 [00:00<00:08,  3.91it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 37 rows, 43 columns and 122 nonzeros
Model fingerprint: 0xe30fb749
Variable types: 6 continuous, 37 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+03, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 690741.00000
Presolve removed 7 rows and 18 columns
Presolve time: 0.00s
Presolved: 30 rows, 25 columns, 100 nonzeros
Variable types: 5 continuous, 20 integer (20 binary)
Found heuristic solution: objective 344599.00000

Root relaxation: objective 1.370557e+05, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 137055.667    0    6 344599.000 137055.667  60.2

  6%|██▍                                         | 2/36 [00:00<00:11,  2.87it/s]

...

 94%|████████████████████████████████████████▌  | 34/36 [00:09<00:00,  4.37it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 127 rows, 43 columns and 410 nonzeros
Model fingerprint: 0xb0246be0
Variable types: 6 continuous, 37 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+03, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 418511.00000
Presolve removed 79 rows and 24 columns
Presolve time: 0.00s
Presolved: 48 rows, 19 columns, 160 nonzeros
Variable types: 5 continuous, 14 integer (14 binary)
Found heuristic solution: objective 252033.00000

Root relaxation: objective 9.257600e+04, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 92576.0000    0    8 252033.000 92576.0000  63.

 97%|█████████████████████████████████████████▊ | 35/36 [00:09<00:00,  5.00it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 97 rows, 43 columns and 314 nonzeros
Model fingerprint: 0x1ee22cf0
Variable types: 6 continuous, 37 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+02, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 301385.00000
Presolve removed 39 rows and 22 columns
Presolve time: 0.00s
Presolved: 58 rows, 21 columns, 176 nonzeros
Variable types: 5 continuous, 16 integer (16 binary)
Found heuristic solution: objective 120313.00000

Root relaxation: objective 1.477000e+03, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1477.00000    0    8 120313.000 1477.00000  98.

100%|███████████████████████████████████████████| 36/36 [00:09<00:00,  3.81it/s]


In [5]:
model_solutions = {}
for i,model in enumerate(primary_models):
    soln = json.loads(model.getJSONSolution())
    model_solutions[i] = soln
json.dump(model_solutions, open('./primary_model_solutions.json', 'w'), indent=4)

In [6]:
s = 0
for model in primary_models:
    s += model.ObjVal
s

8138571.0

In [7]:
cluster
Neighbours

[109, 114, 141, 170, 174]

In [8]:
json.loads(model.getJSONSolution())

{'SolutionInfo': {'Status': 2,
  'Runtime': 0.1295638084411621,
  'Work': 0.0032717434235194722,
  'ObjVal': 120313,
  'ObjBound': 120313,
  'ObjBoundC': 120313,
  'MIPGap': 0,
  'IntVio': 0,
  'BoundVio': 0,
  'ConstrVio': 0,
  'IterCount': 68,
  'BarIterCount': 0,
  'NodeCount': 1,
  'SolCount': 2,
  'PoolObjBound': 120313,
  'PoolObjVal': [120313, 301385]},
 'Vars': [{'VarName': 'x[0,1]', 'X': 1},
  {'VarName': 'x[0,2]', 'X': 1},
  {'VarName': 'x[1,3]', 'X': 1},
  {'VarName': 'x[2,4]', 'X': 1},
  {'VarName': 'x[3,5]', 'X': 1},
  {'VarName': 'x[4,0]', 'X': 1},
  {'VarName': 'x[5,0]', 'X': 1},
  {'VarName': 'L', 'X': 2},
  {'VarName': 'u[1]', 'X': 2},
  {'VarName': 'u[2]', 'X': 4},
  {'VarName': 'u[3]', 'X': 3},
  {'VarName': 'u[4]', 'X': 5},
  {'VarName': 'u[5]', 'X': 6}]}